In [1]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



**Input**

In [2]:
ls -d ${FD_WRK}/data/*/

/work/kk319/out/CombEffect_STARR/data/Input/
/work/kk319/out/CombEffect_STARR/data/Input1/
/work/kk319/out/CombEffect_STARR/data/Input2/
/work/kk319/out/CombEffect_STARR/data/Input3/
/work/kk319/out/CombEffect_STARR/data/Input4/
/work/kk319/out/CombEffect_STARR/data/Input5/
/work/kk319/out/CombEffect_STARR/data/TFX2_DMSO/
/work/kk319/out/CombEffect_STARR/data/TFX2_Dex/
/work/kk319/out/CombEffect_STARR/data/TFX3_DMSO/
/work/kk319/out/CombEffect_STARR/data/TFX3_Dex/
/work/kk319/out/CombEffect_STARR/data/TFX4_DMSO/
/work/kk319/out/CombEffect_STARR/data/TFX4_Dex/
/work/kk319/out/CombEffect_STARR/data/TFX5_DMSO/
/work/kk319/out/CombEffect_STARR/data/TFX5_Dex/
/work/kk319/out/CombEffect_STARR/data/TFX_DMSO/
/work/kk319/out/CombEffect_STARR/data/TFX_Dex/


In [3]:
ls ${FD_WRK}/data/Input/

ann_chr17.bed        chr1.bed   chr14.bed  chr19.bed  chr3.bed  chr8.bed
ann_chr17_CREB1.bed  chr10.bed  chr15.bed  chr2.bed   chr4.bed  chr9.bed
ann_chr17_FOS.bed    chr11.bed  chr16.bed  chr20.bed  chr5.bed  chrX.bed
ann_chr17_GCR.bed    chr12.bed  chr17.bed  chr21.bed  chr6.bed  chrY.bed
ann_chr17_JUN.bed    chr13.bed  chr18.bed  chr22.bed  chr7.bed  target_PER1.bed


## Count fragments for each chromosome

**TEST**

In [7]:
CHROMS=($(seq 1 22) X Y)
for idx in {0..23}; do
    echo "Index: $idx; Chromosome ${CHROMS[$idx]}"
done

Index: 0; Chromosome 1
Index: 1; Chromosome 2
Index: 2; Chromosome 3
Index: 3; Chromosome 4
Index: 4; Chromosome 5
Index: 5; Chromosome 6
Index: 6; Chromosome 7
Index: 7; Chromosome 8
Index: 8; Chromosome 9
Index: 9; Chromosome 10
Index: 10; Chromosome 11
Index: 11; Chromosome 12
Index: 12; Chromosome 13
Index: 13; Chromosome 14
Index: 14; Chromosome 15
Index: 15; Chromosome 16
Index: 16; Chromosome 17
Index: 17; Chromosome 18
Index: 18; Chromosome 19
Index: 19; Chromosome 20
Index: 20; Chromosome 21
Index: 21; Chromosome 22
Index: 22; Chromosome X
Index: 23; Chromosome Y


In [12]:
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    echo "SAMPLE: ${SAMPLE}; FD_BED: ${FD_BED}"
done

SAMPLE: Input; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input/
SAMPLE: Input1; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input1/
SAMPLE: Input2; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input2/
SAMPLE: Input3; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input3/
SAMPLE: Input4; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input4/
SAMPLE: Input5; FD_BED: /work/kk319/out/CombEffect_STARR/data/Input5/
SAMPLE: TFX2_DMSO; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX2_DMSO/
SAMPLE: TFX2_Dex; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX2_Dex/
SAMPLE: TFX3_DMSO; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX3_DMSO/
SAMPLE: TFX3_Dex; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX3_Dex/
SAMPLE: TFX4_DMSO; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX4_DMSO/
SAMPLE: TFX4_Dex; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX4_Dex/
SAMPLE: TFX5_DMSO; FD_BED: /work/kk319/out/CombEffect_STARR/data/TFX5_DMSO/
SAMPLE: TFX5_Dex; FD_BED: /work/kk319/out/CombEffect_STA

**RUN**

In [41]:
source ../config_duke.sh

sbatch -p scavenger \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/count_fragment_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=chr${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
FD_OUT=${FD_WRK}/count_fragment
FN_BED=${TARGET}.bed

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_CNT=${FD_OUT}/${SAMPLE}
    FN_CNT=${TARGET}.bed
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_CNT}/${FN_CNT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_CNT}
    
    ### count the fragments and arrange output to a proper bed file
    cat ${FD_BED}/${FN_BED} |\
        sort    |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        sort -k2,2n > ${FD_CNT}/${FN_CNT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_CNT}/${FN_CNT}
    head -n 3 ${FD_CNT}/${FN_CNT}
    echo
done
echo "Done!"

EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Submitted batch job 14789207


**CHECK**

In [43]:
ls ${FD_LOG}/count_fragment_chrom.17.txt

/work/kk319/out/CombEffect_STARR/log/count_fragment_chrom.17.txt


In [44]:
cat ${FD_LOG}/count_fragment_chrom.17.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  17
RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
Output file:  /work/kk319/out/CombEffect_STARR/count_fragment/Input/chr18.bed

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
chr18	21483	22446
chr18	21483	22446
chr18	28214	29137

Show the first few lines of the output file
/work/kk319/out/CombEffect_STARR/count_fragment/Input/chr18.bed
chr18	15221	16301	1
chr18	20556	21638	1
chr18	20605	21575	1

++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input1//chr18.bed
Output f